In [72]:
# Import all the libraries
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms as TF
import cv2
import numpy as np
import glob
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from transformers import ViTForImageClassification, ViTFeatureExtractor
from PIL import Image

In [76]:
# make a df out of the test folder
final_df = pd.DataFrame(columns=['filename', 'label'])

for folder_name in glob.glob(r'D:\College\Semester_2\CT5135\Group_8_wet_slight\test'+'/*'):
    for filename in glob.glob(folder_name+'/*'):
        if filename.endswith('.jpg'):
            final_df = pd.concat([final_df, pd.DataFrame({'filename': [filename], 'label': [os.path.basename(folder_name)]})])

final_df.reset_index(drop=True, inplace=True)
final_df['numerical_labels'] = pd.Categorical(final_df['label']).codes
print(final_df)

                                               filename               label  \
0     D:\College\Semester_2\CT5135\Group_8_wet_sligh...  wet_asphalt_slight   
1     D:\College\Semester_2\CT5135\Group_8_wet_sligh...  wet_asphalt_slight   
2     D:\College\Semester_2\CT5135\Group_8_wet_sligh...  wet_asphalt_slight   
3     D:\College\Semester_2\CT5135\Group_8_wet_sligh...  wet_asphalt_slight   
4     D:\College\Semester_2\CT5135\Group_8_wet_sligh...  wet_asphalt_slight   
...                                                 ...                 ...   
5495  D:\College\Semester_2\CT5135\Group_8_wet_sligh...          wet_gravel   
5496  D:\College\Semester_2\CT5135\Group_8_wet_sligh...          wet_gravel   
5497  D:\College\Semester_2\CT5135\Group_8_wet_sligh...          wet_gravel   
5498  D:\College\Semester_2\CT5135\Group_8_wet_sligh...          wet_gravel   
5499  D:\College\Semester_2\CT5135\Group_8_wet_sligh...          wet_gravel   

      numerical_labels  
0                    0  
1

In [77]:
# Create an empty DataFrame with 6 columns
empty_df = pd.DataFrame(columns=['class_1_True', 'class_1_False', 'class_2_True', 'class_2_False', 'class_3_True', 'class_3_False'])

result_df = pd.concat([final_df, empty_df], axis=1)

In [78]:
result_df.drop(columns=['label'], inplace=True)

In [79]:
result_df.head()

,filename,numerical_labels,class_1_True,class_1_False,class_2_True,class_2_False,class_3_True,class_3_False
0,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,NaN,NaN,NaN,NaN,NaN,NaN
1,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,NaN,NaN,NaN,NaN,NaN,NaN
2,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,NaN,NaN,NaN,NaN,NaN,NaN
3,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,NaN,NaN,NaN,NaN,NaN,NaN
4,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# Load the three models 
transform = TF.Compose([
    TF.ToPILImage(),
    TF.Resize((224, 224)),
    TF.ToTensor(),
    TF.Normalize(mean=[0.4672, 0.4943, 0.4919],std=[0.1296, 0.1288, 0.1283])
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [93]:
# import our fine-tuned model
model_name_or_path = r'D:\College\Semester_2\CT5135\checkpoint\checkpoint-17000'

model_1 = ViTForImageClassification.from_pretrained(model_name_or_path)
feature_extractor_finetuned = ViTFeatureExtractor.from_pretrained(model_name_or_path)
model_1.to(device)

for index, row in result_df.iterrows():
    if index % 100 == 0:
        print(index)
    image = Image.open(row['filename'])
    inputs = feature_extractor_finetuned(image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        logits = model_1(**inputs).logits
    p = nn.functional.softmax(logits, dim=1)
    # print(p[:, 0].item(), p[:, 1].item())
    result_df.loc[index, 'class_1_True'] = p[:, 1].item()
    result_df.loc[index, 'class_1_False'] = p[:, 0].item()
    result_df.loc[index, 'class_1_binary'] =logits.argmax(-1).item() 


c:\Users\Tarun\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


0
0.001280923024751246 0.9987190961837769
0.004841235466301441 0.9951587319374084
0.001279412186704576 0.9987205862998962
0.0012988619273528457 0.9987010955810547
0.0013331921072676778 0.9986667633056641
0.0013511967845261097 0.9986488223075867
0.0014511323533952236 0.9985488057136536
0.0012760796817019582 0.9987239241600037
0.001280817436054349 0.9987192153930664
0.0012834705412387848 0.9987165927886963
0.001290963846258819 0.9987090826034546
0.001442070584744215 0.998557984828949
0.0012794415233656764 0.9987205862998962
0.00145351595710963 0.9985464215278625
0.001363960443995893 0.9986360669136047
0.0013110702857375145 0.9986889958381653
0.0012932743411511183 0.9987066984176636
0.001410839264281094 0.9985890984535217
0.0013523001689463854 0.9986477494239807
0.0012962075415998697 0.9987038373947144
0.0013006062945351005 0.998699426651001
0.0015565359499305487 0.9984434247016907
0.002893111202865839 0.9971068501472473
0.0012816368835046887 0.9987183809280396
0.0013011625269427896 0.998

In [95]:
model_2 = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
model_2.load_state_dict(torch.load(r'D:\College\Semester_2\CT5135\Model_2_best_model.pth')) 

model_2.to(device)
model_2.eval()

for index, row in result_df.iterrows():
    if index % 100 == 0:
        print(index)
    image = cv2.imread(row['filename'])
    image = transform(image)
    image = image.to(device) 
    with torch.no_grad():
        logit = model_2(image.unsqueeze(0))
    p = nn.functional.softmax(logit, dim=1)
    result_df.loc[index, 'class_2_True'] = p[:, 1].item()
    result_df.loc[index, 'class_2_False'] = p[:, 0].item()
    result_df.loc[index, 'class_2_binary'] = nn.functional.softmax(logit, dim=1).argmax(1).item()


c:\Users\Tarun\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tarun\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [100]:
model_3 = models.resnet50(pretrained=True)
model_3.load_state_dict(torch.load(r'D:\College\Semester_2\CT5135\Model_3_best_model.pth')) 
model_3.to(device)
model_3.eval()


for index, row in result_df.iterrows():
    if index % 100 == 0:
        print(index)
    image = cv2.imread(row['filename'])
    image = transform(image)
    image = image.to(device) 
    with torch.no_grad():
        logit = model_3(image.unsqueeze(0))
    p = nn.functional.softmax(logit, dim=1)
    result_df.loc[index, 'class_3_True'] = p[:, 0].item()
    result_df.loc[index, 'class_3_False'] = p[:, 1].item()
    result_df.loc[index, 'class_3_binary'] = 1 if nn.functional.softmax(logit, dim=1).argmax(1).item() == 0.0 else 0 


c:\Users\Tarun\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tarun\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400


In [101]:
result_df.to_csv('results.csv', index=False)

In [102]:
result_df.head()

,filename,numerical_labels,class_1_True,class_1_False,class_2_True,class_2_False,class_3_True,class_3_False,class_1_binary,class_2_binary,class_3_binary
0,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,0.998719,0.001281,0.0,0.99973,0.00032,0.999618,1.0,0.0,0.0
1,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,0.995159,0.004841,0.649923,0.339579,0.358195,0.641588,1.0,1.0,0.0
2,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,0.998721,0.001279,0.0,0.999715,0.0,0.999999,1.0,0.0,0.0
3,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,0.998701,0.001299,0.0,0.999923,0.000055,0.999925,1.0,0.0,0.0
4,D:\College\Semester_2\CT5135\Group_8_wet_sligh...,0,0.998667,0.001333,0.000117,0.999262,0.000095,0.999793,1.0,0.0,0.0


In [105]:
# Split the dataset into test train and validation

X = result_df.drop(columns=['numerical_labels','filename'])  
y = result_df['numerical_labels']

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [118]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [119]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [2, 4, 8,16],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}

rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 4, 8, 16],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=2)

In [120]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}


In [121]:
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=8, min_samples_split=10, random_state=42)

In [130]:
results = best_rf_model.predict(X_val)

In [131]:
print(results)

[1 2 0 0 0 2 2 2 0 0 1 0 0 2 2 2 0 1 0 1 0 0 0 2 1 0 2 0 0 0 0 2 1 1 2 0 2
 2 0 0 2 2 0 0 0 1 0 0 0 0 0 0 2 1 0 2 0 0 0 1 2 0 2 1 0 2 0 2 0 2 2 2 2 0
 2 0 1 1 0 0 2 0 0 2 0 2 2 0 0 1 2 0 2 2 2 2 0 1 2 2 1 0 2 2 0 0 2 1 0 0 2
 0 0 0 0 2 2 0 1 1 0 0 2 2 1 2 0 2 2 2 0 0 0 0 2 2 0 2 0 2 2 0 2 0 1 1 2 0
 2 2 0 0 2 2 0 0 2 0 2 1 0 2 2 2 0 0 0 2 0 2 2 2 1 2 0 0 2 0 0 1 2 0 0 2 2
 2 0 0 0 0 1 1 2 2 2 0 0 0 0 2 0 2 2 2 2 0 0 0 0 2 0 0 0 0 2 2 0 2 1 0 2 1
 1 2 0 0 0 2 1 2 0 2 2 0 2 2 2 2 2 2 2 0 2 2 0 0 2 2 0 0 2 2 2 0 0 2 1 0 0
 0 0 2 0 0 2 2 2 2 0 2 2 1 2 2 0 2 1 1 0 0 0 2 0 2 0 0 1 1 0 2 0 2 0 2 0 0
 0 0 1 0 0 2 1 0 0 0 0 2 2 1 0 0 2 0 1 2 1 1 0 2 0 0 0 0 0 0 0 1 0 2 2 2 2
 2 0 1 2 1 0 0 0 0 0 1 2 0 2 0 0 2 2 2 0 1 2 0 0 0 0 2 0 0 0 0 2 0 0 2 2 2
 0 2 0 0 0 0 2 2 0 1 2 2 2 0 0 0 0 2 0 2 0 1 2 2 2 0 2 2 0 2 1 2 2 1 0 2 2
 0 2 2 2 0 2 2 0 2 0 0 2 1 0 2 2 2 0 0 0 0 2 2 2 2 2 0 0 2 0 0 2 0 2 2 0 1
 1 2 0 1 0 2 2 0 2 0 1 0 2 2 2 0 2 0 0 0 2 0 0 2 2 0 2 2 0 0 0 0 2 0 0 0 1
 0 0 0 2 2 2 0 2 2 2 2 0 

In [122]:
val_accuracy = best_rf_model.score(X_val, y_val)
print("Validation Accuracy:", val_accuracy)


Validation Accuracy: 0.9793939393939394


In [125]:
test_accuracy = best_rf_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.9793939393939394


In [132]:
# Save the model
import joblib
joblib.dump(best_rf_model, 'best_rf_model.joblib')


['best_rf_model.joblib']